## **Análise de Sentimentos usando Machine Learning**

* Criando modelos para análise de sentimentos de tweets
* Teste com Modelo usando tag de negações
* Teste com Modelo usando Bigrams

In [2]:
from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Ler arquivo de dados e conta a quantidade de linhas**

In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tweets_Mg.csv' )

**Exibe as 50 primeiras lihas de tweets**

In [ ]:
dataset.head(50)

**Conta a quantidade de linhas de tweets neutros, positivos e negativos**

In [ ]:
dataset[dataset.Classificacao=='Neutro'].count()

In [ ]:
dataset[dataset.Classificacao=='Positivo'].count()

In [ ]:
dataset[dataset.Classificacao=='Negativo'].count()

In [ ]:
%matplotlib inline
dataset.Classificacao.value_counts().plot(kind='bar')

In [ ]:
dataset.count()

## Pre-Processamento dos Dados

* Remove linhas duplicadas na base de dados
    - Problema na coleta dos dados.
* Remove Stopwords
* Faz Stemming nos dados
* Remove caracteres indesejados como links, pontuação etc.

In [ ]:
dataset.drop_duplicates(['Text'], inplace=True)

In [ ]:
dataset.Text.count()

## ** Separando tweets e suas Classes**

In [ ]:
tweets = dataset['Text']
classes = dataset['Classificacao']

** Instala bibliotecas e baixa a base de dados**

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')

**Funções de Pre-processamento de dados**

In [ ]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [ ]:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

In [ ]:
def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

** Entenda como funciona cada função**

In [ ]:
RemoveStopWords('Eu não gosto do partido, e também não votaria novamente nesse governante!')

In [ ]:
Stemming('Eu não gosto do partido, e também não votaria novamente nesse governante!')

In [ ]:
Limpeza_dados('Assita aqui o video do Governador falando sobre a CEMIG https://www.uol.com.br :) ;)')

** Aplica as 3 funções de Pre-processamento nos dados**

In [ ]:
def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [stemmer.stem(i) for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Aplica a função em todos os dados:
tweets = [Preprocessing(i) for i in tweets]

In [ ]:
Preprocessing('Eu não gosto do partido, e também não votaria novamente nesse governante. Assita o video aqui https:// :)')

**Visualize os dados e veja como ficou após o pré-processamento**

In [ ]:
tweets[:50]

## Criando o modelo

**Instancia o objeto que faz a vetorização dos dados de texto**

In [ ]:
vectorizer = CountVectorizer(analyzer="word")

**Aplica o vetorizador nos dados de texto**

In [ ]:
freq_tweets = vectorizer.fit_transform(tweets)
type(freq_tweets)

In [ ]:
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

**Formato (Linhas, Colunas) da matriz**

In [ ]:
freq_tweets.shape

**Matriz**

In [ ]:
freq_tweets.A

** Testando o modelo com algumas instâncias simples**

In [ ]:
# defina instâncias de teste dentro de uma lista
testes = ['Esse governo está no início, vamos ver o que vai dar',
          'Estou muito feliz com o governo de Minas esse ano',
          'O estado de Minas Gerais decretou calamidade financeira!!!',
          'A segurança desse país está deixando a desejar',
          'O governador de Minas é mais uma vez do PT']

**Aplica a função de Pré-processamento nos dados**

In [ ]:
testes = [Preprocessing(i) for i in testes]

In [ ]:
# Transforma os dados de teste em vetores de palavras.
freq_testes = vectorizer.transform(testes)

In [ ]:
# Fazendo a classificação com o modelo treinado.
for t, c in zip (testes,modelo.predict(freq_testes)):
    print (t +", "+ c)

In [ ]:
# Probabilidades de cada classe
print (modelo.classes_)
modelo.predict_proba(freq_testes).round(2)

## ** Função de Tags de Negações**

* Acrescenta uma tag _NEG encontrada após um 'não'.
* Objetivo é dar mais peso para o modelo identificar uma inversão de sentimento da frase.
* Exemplos: 
    - Eu gosto de cachorros, positivo.
    - Eu **não** gosto de cachorros, negativo.

In [ ]:
def marque_negacao(texto):
    negacoes = ['não','not']
    negacao_detectada = False
    resultado = []
    palavras = texto.split()
    for p in palavras:
        p = p.lower()
        if negacao_detectada == True:
            p = p + '_NEG'
        if p in negacoes:
            negacao_detectada = True
        resultado.append(p)
    return (" ".join(resultado))

**Exemplos de utilização da tag de negações**

In [ ]:
marque_negacao('Eu gosto do partido, votaria novamente nesse governante!')

In [ ]:
marque_negacao('Eu Não gosto do partido, e também não votaria novamente nesse governante!')

## **Criando modelos com Pipelines**

* Pipelines são interessantes para reduzir código e automatizar fluxos

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline_simples = Pipeline([
  ('counts', CountVectorizer()),
  ('classifier', MultinomialNB())
])

* Pipeline que atribui tag de negacoes nas palavras

In [ ]:
pipeline_negacoes = Pipeline([
  ('counts', CountVectorizer(tokenizer=lambda text: marque_negacao(text))),
  ('classifier', MultinomialNB())
])

In [ ]:
pipeline_simples.fit(tweets,classes)

In [ ]:
pipeline_simples.steps

* Gera o modelo de negações

In [ ]:
pipeline_negacoes.fit(tweets,classes)

* Etapas do pipeline

In [ ]:
pipeline_negacoes.steps

## Validando os Modelos com Validação Cruzada

* Fazendo o cross validation do modelo

In [ ]:
resultados = cross_val_predict(pipeline_simples, tweets, classes, cv=10)

* Medindo a acurácia média do modelo

In [ ]:
metrics.accuracy_score(classes,resultados)

* Medidas de validação do modelo

In [ ]:
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

* Matriz de confusão

In [ ]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

## **Modelo com a Tag de Negações**

In [ ]:
resultados = cross_val_predict(pipeline_negacoes, tweets, classes, cv=10)

* Medindo a acurácia média do modelo

In [ ]:
metrics.accuracy_score(classes,resultados)

In [ ]:
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

* Matriz de confusão

In [ ]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

## ** Avaliando modelo com Bigrams**

In [ ]:
'eu gosto', 'gosto do' , 'do brasil'

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2))
freq_tweets = vectorizer.fit_transform(tweets)
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

In [ ]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [ ]:
metrics.accuracy_score(classes,resultados)

In [ ]:
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

In [ ]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

## ** Considerações Finais**

* Considere aumentar a quantidade de dados de treino.

* Pela sua simplicidade o Naive Bayes pode ser usado perfeitamente como um algoritmo de Baseline.

* Considere alterar os parâmetros do algoritmo.

**www.minerandodados.com.br** - *A maior comunidade de Data Science do Brasil*